In [1]:
import pymupdf4llm

my_cv_path = "/Users/rsukumar/Downloads/Sukumar_RAGHAVAN_CV.pdf"

# Load pdf file
llama_reader = pymupdf4llm.LlamaMarkdownReader()
llama_docs = llama_reader.load_data(my_cv_path)

Successfully imported LlamaIndex
Processing /Users/rsukumar/Downloads/Sukumar_RAGHAVAN_CV.pdf...
]
Processing /Users/rsukumar/Downloads/Sukumar_RAGHAVAN_CV.pdf...
]========================================] (1/1)


In [14]:
from IPython.display import Markdown, display

import chromadb
from chromadb.utils.embedding_functions import create_langchain_embedding
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

# Using Huggingface embeddings
langchain_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
ef = create_langchain_embedding(langchain_embeddings)
embed_model = LangchainEmbedding(langchain_embeddings)

# Keeping option to switch between Ephemeral / Persistent chroma client
chroma_persist_path = "chroma-db-job-coverletter"
vector_db_collection_name = "job-cv-vector-db-collection"
# ephemeral_chroma_client = chromadb.EphemeralClient()
persistent_chroma_client = chromadb.PersistentClient(path=chroma_persist_path)
vector_db_client = persistent_chroma_client

chroma_collection = vector_db_client.get_or_create_collection(
    vector_db_collection_name, embedding_function=ef
)

vector_store_from_client = Chroma(
    client=vector_db_client,
    collection_name=vector_db_collection_name,
    embedding_function=ef,
)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    llama_docs,
    storage_context=storage_context,
    embed_model=embed_model,
)

# Query Data
query_engine = index.as_query_engine(llm="local", streaming=True)
response = query_engine.query("What is the document all about?")
display(Markdown(f"<b>{response}</b>"))

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/rsukumar/Library/Caches/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:              

<b>  Based on the provided context information, the document appears to be a curriculum vitae (CV) or a resume of a person named Sukumar Raghavan. The document contains information about his educational background, work experience, and skills. The document is in PDF format, and it was created on December 11, 2024, at 5:32 AM. The document has 2 pages, and the file path is /Users/rsukumar/Downloads/Sukumar_RAGHAVAN_CV.pdf.</b>

In [18]:
from IPython.display import Markdown, display

# Use local Ollama model to answer the question
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOllama

# Prompt template
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer just say I don't know, don't try to make up an answer.
{context}
Question: {question}
Answer as a jobseeker:"""

question = "Write a cover letter for applying to the job."
llm = ChatOllama(model="llama3.2", temperature=0)
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

chroma_db_persist_path = "chroma-db-job-coverletter"
vector_db_collection_name = "job-cv-vector-db-collection"
lc_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
ef2 = create_langchain_embedding(lc_embeddings)

vector_db = Chroma(
    collection_name="sukumar-cv-vector-db-collection",
    persist_directory=chroma_db_persist_path,
    embedding_function=ef2,
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vector_db.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    # chain_type="map_reduce", # other techniques to experiment: "refine", "map_rerank"
)
result = qa_chain({"query": question})

display(Markdown(f"<b>{result["result"]}</b>"))

/var/folders/g1/7p_ff_xn2ms3b005zgc5tbrc0000gn/T/ipykernel_79326/1785563842.py:35: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


<b>I'd be happy to help with that. Here's a sample cover letter:

[Your Name]
[Address]
[City, State ZIP Code]
[Date]

[Hiring Manager's Name]
[Company Name]
[Address]
[City, State ZIP Code]

Dear [Hiring Manager's Name],

I am excited to apply for the [Job Title] position at [Company Name]. With [Number of Years] years of experience in [Industry/Field], I am confident that my skills and qualifications make me an ideal candidate for this role.

As a highly motivated and dedicated professional, I have a proven track record of [Desirable Skill or Achievement]. In my current role at [Current Company], I have successfully [Briefly describe a relevant accomplishment or experience]. I am excited about the opportunity to bring my skills and experience to a new challenge and contribute to the success of [Company Name].

I am particularly drawn to this role because of [Reason for interest in company or position]. I am impressed by [Company's Mission or Values] and believe that my own values and work style align with those of the organization.

Thank you for considering my application. I would welcome the opportunity to discuss this position further and explain in greater detail why I am the ideal candidate for this role. Please feel free to contact me at [Your Contact Information].

Sincerely,

[Your Name]

Is there anything else I can help you with?</b>